In [1]:
import sys
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
# import os
# from google.colab import files
# uploaded = files.upload()
# dst ='image.jpg' 
# os.rename(list(uploaded.keys())[0], dst)

In [3]:
# import os
# from google.colab import files
# uploaded = files.upload()
# dst ='movie.mp4' 
# os.rename(list(uploaded.keys())[0], dst)

In [4]:
# damage colors(BGR)
elementColors = dict()
elementColors['anemo'] = (200, 255, 106)
elementColors['geo'] = (101, 204, 254)
elementColors['electro'] = (255, 155, 224)
elementColors['physical'] = (255, 255, 255)
elementColors['pyro'] = (3, 155, 254)
elementColors['cyro'] = (255, 252, 155)
elementColors['hydro'] = (253, 204, 52)

resizeScale = 0.5
colorKey = "cyro"

In [8]:
imageCounter = 0
def outputNumberBinaryImage(im, rect):
  global imageCounter
  x,y,w,h = rect
  outputImage = im[y:y+h, x:x+w]
  closing = cv2.dilate(outputImage,(4, 4),iterations = 1)
  cv2.imwrite("./output/{}_{}.png".format(colorKey, imageCounter), closing)
  imageCounter += 1

In [9]:
def rectangleScaled(im, start, end, color, thickness=1):
    start = (int(start[0] / resizeScale), int(start[1] / resizeScale))
    end = (int(end[0] / resizeScale), int(end[1] / resizeScale))
    cv2.rectangle(im, start, end, color, thickness)

In [10]:


def drawContours(im, resized, parcent, width, height):
  global prevFrameNumber, resizeScale, textpos
  
  rangeW = 25
  color = elementColors[colorKey]
  inrange = cv2.inRange(resized, (max(color[0] - rangeW, 0), max(color[1] - rangeW, 0), max(color[2] - rangeW, 0)), (min(255, color[0] + rangeW), min(255, color[1] + rangeW), min(255, color[2] + rangeW)))
  # for key, color in elementColors.items():
  #   nextInrange = cv2.inRange(im, (color[0] - rangeW, color[1] - rangeW, color[2] - rangeW), (color[0] + rangeW, color[1] + rangeW, color[2] + rangeW))
  #   inrange = np.maximum(nextInrange, inrange)

  # blur = cv2.GaussianBlur(inrange, (5, 5), 0)
  thresh = cv2.adaptiveThreshold(inrange, 255, 1, 1, 7, 2)

  contours, _ = cv2.findContours(
      thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  
  frameContours = []
  
  nextFrameNumber = []

  for cnt in contours:
    rect = cv2.boundingRect(cnt.astype(np.int32))
    x, y, w, h = rect
    frameContours.append((x + w / 2, y + h / 2))
    

    if h < 2 or w / (h * 0.8) < 3:
      continue
    
    outputNumberBinaryImage(inrange, rect)
    red = (0, 0, 255)
    rectangleScaled(im, (x, y), (x + w, y + h), red, 2)


  return im

In [14]:
filename = "rasen"
cap = cv2.VideoCapture('./{}.mp4'.format(filename))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./{}_output.mp4'.format(filename), fourcc, frame_rate, (width, height))
colorKey = "cyro"
prevFrameNumber = []
import queue
textQueue = queue.Queue()

count = 0
imageCounter = 0

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

while(cap.isOpened()):
  ret, frame = cap.read()
  count += 1
  if count % 24 == 0:
    print(count / frame_count * 100, '%')
  if (not(ret)):
    break
  
  color = elementColors[colorKey]
  rangeW = 30
  resized = cv2.resize(frame, (int(frame.shape[1] * resizeScale), int(frame.shape[0] * resizeScale)))
  res = drawContours(frame, resized, count / frame_count, frame.shape[1], frame.shape[0])

    
  # inrange = cv2.inRange(resized, (max(color[0] - rangeW, 0), max(color[1] - rangeW, 0), max(color[2] - rangeW, 0)), (min(255, color[0] + rangeW), min(255, color[1] + rangeW), min(255, color[2] + rangeW)))
  # # for key, color in elementColors.items():
  # #   nextInrange = cv2.inRange(im, (color[0] - rangeW, color[1] - rangeW, color[2] - rangeW), (color[0] + rangeW, color[1] + rangeW, color[2] + rangeW))
  # #   inrange = np.maximum(nextInrange, inrange)

  # # blur = cv2.GaussianBlur(inrange, (5, 5), 0)
  # res = cv2.adaptiveThreshold(inrange, 255, 1, 1, 7, 2)

  # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  out.write(res)
  cv2.imshow("frame", res)
  
  if cv2.waitKey(1) & 0xFF == ord('q'):
      break

out.release()
cap.release()
cv2.destroyAllWindows()
    


3.5555555555555554 %
7.111111111111111 %
10.666666666666668 %
14.222222222222221 %
17.77777777777778 %
21.333333333333336 %
24.88888888888889 %
28.444444444444443 %
32.0 %
35.55555555555556 %
39.111111111111114 %
42.66666666666667 %
46.22222222222222 %
49.77777777777778 %
53.333333333333336 %
56.888888888888886 %
60.44444444444444 %
64.0 %
67.55555555555556 %
71.11111111111111 %
74.66666666666667 %
78.22222222222223 %
81.77777777777779 %
85.33333333333334 %
88.88888888888889 %
92.44444444444444 %
96.0 %
99.55555555555556 %


In [24]:
import os
image_fns = os.listdir('./output')
for i in range(len(image_fns)):
  os.rename(''.join(['./output/', image_fns[i]]), ''.join(['./output2/', str(i), '.png']))


In [29]:
import cv2
image_fns = os.listdir('./output2')
for i in range(len(image_fns)):
  img = cv2.imread(''.join(['./output2/', image_fns[i]]))
  height, width, _ = img.shape
  img = cv2.resize(img, (200, 50))
  cv2.imwrite(''.join(['./output3/', image_fns[i]]), img)


0.png
1.png
10.png
100.png
101.png
102.png
103.png
104.png
105.png
106.png
107.png
108.png
109.png
11.png
110.png
111.png
112.png
113.png
114.png
115.png
116.png
117.png
118.png
119.png
12.png
120.png
121.png
122.png
123.png
124.png
125.png
126.png
127.png
128.png
129.png
13.png
130.png
131.png
132.png
133.png
134.png
135.png
136.png
137.png
138.png
139.png
14.png
140.png
141.png
142.png
143.png
144.png
145.png
146.png
147.png
148.png
149.png
15.png
150.png
151.png
152.png
153.png
154.png
155.png
156.png
157.png
158.png
159.png
16.png
160.png
161.png
162.png
17.png
18.png
19.png
2.png
20.png
21.png
22.png
23.png
24.png
25.png
26.png
27.png
28.png
29.png
3.png
30.png
31.png
32.png
33.png
34.png
35.png
36.png
37.png
38.png
39.png
4.png
40.png
41.png
42.png
43.png
44.png
45.png
46.png
47.png
48.png
49.png
5.png
50.png
51.png
52.png
53.png
54.png
55.png
56.png
57.png
58.png
59.png
6.png
60.png
61.png
62.png
63.png
64.png
65.png
66.png
67.png
68.png
69.png
7.png
70.png
71.png
72.png
73.png
